In [2]:
class BankAccount:
    def __init__(self, account_number, name, password, balance, account_type):
        self._id = account_number
        self._name = name
        self._balance = balance
        self._password = password
        self._account_type = account_type
        self._interest_rate = 8

    def check_balance(self):
        return self._balance
    def is_valid_transact_amount(self, amount):
        if(amount > 0):
            return True
        raise Exception('Invalid Amount. Amount should be greater than 0')
    
    def withdraw(self, amount):
        if self.is_valid_transact_amount(amount) :
            self._balance -= amount
            return True
    
    def deposit(self, amount):
        if self.is_valid_transact_amount(amount):
            self._balance += amount
            return True

    def credit_interest(self):
        if not self._balance<=0:
            interest = self._balance * self._interest_rate/1200
            self.deposit(interest)

    def get_info(self):
        return f'{self._name} {self._balance}'

class SavingsAccount(BankAccount):
    def __init__(self, account_number, name, password, balance, min_balance=5000):
        super().__init__(account_number, name, password, balance, account_type='SAVING')
        self._interest_rate = 12
        self._min_balance = min_balance
    
    
    def get_max_withdrawal_amount(self):
        return self._balance - self._min_balance
    
    def withdraw(self, amount):
        if amount < self.get_max_withdrawal_amount(): 
            return super().withdraw(amount)
    

class CurrentAccount(BankAccount):
    def __init__(self, account_number, name, password, balance):
        super().__init__(account_number, name, password, balance, account_type='CURRENT')
        self._interest_rate = 0
        self._min_balance = 0
    
class OverDraftAccount(BankAccount):
    def __init__(self, account_number, name, password, balance):
        super().__init__(account_number, name, password, balance, account_type='OVERDRAFT')
        self._interest_rate = 8
        self._max_balance = 0
        self._od_limit = 0
        self._od_fee_interest = 1


    def get_od_limit(self):
        limit = self._max_balance / 10
        return limit
    
    def get_max_withdrawal_amount(self):
        return self._balance + self.get_od_limit()
    
    def calculate_od_fee(self, amount):
        return (amount - self._balance)/100
    
    def withdraw(self, amount):
        if(amount < self.get_max_withdrawal_amount()):
            od_fee = self.calculate_od_fee(amount)
            super().withdraw(amount)
            super().withdraw(od_fee)
            return True
        return False
    
    def deposit(self, amount):
        super().deposit(amount)
        if self._balance > self._max_balance:
            self._max_balance = self._balance
    



class Bank:
    def __init__(self):
        self.__accounts = []
        self.__last_id = 1

    def is_valid_account(self, account):
        return isinstance(account, BankAccount)

    def create_account(self, name,password, balance = 0,min_balance = 0 , account_type='SAVING'):
        if account_type.upper()=='CURRENT' :
            account = CurrentAccount(self.__last_id, name, password, balance)

        elif account_type.upper()=='OVERDRAFT' :
            account = OverDraftAccount(self.__last_id, name, password, balance)

        else:
            account = SavingsAccount(self.__last_id, name, password, balance, min_balance)

        self.__accounts.append(account)
        self.__last_id += 1
        return account

    def authenticate(self, account, password):
        if self.is_valid_account(account):
            return account._password == password
        raise Exception('Invalid Credentials')
    
    def get_account(self, account_number):
        for index, account in enumerate(self.__accounts):
            if account._id == account_number:
                return (index, account)
        
        raise Exception('Account does not exist')
    
    def get_account_type(self, account_type):
        if(account_type == 'SAVING'):
            return SavingsAccount
        elif account_type == 'CURRENT':
            return CurrentAccount   
        elif account_type == 'OVERDRAFT':
            return OverDraftAccount
        else:
            return None

    def delete_account(self, account_number):
        index, account = self.get_account(account_number)
        self.__accounts.pop(index)

    def transfer_money(self, from_account, to_account, amount):
        if not self.is_valid_account(from_account) or not self.is_valid_account(to_account): 
            raise Exception('Invalid account object')
        if from_account.withdraw(amount):
            to_account.deposit(amount)
            return True
        raise Exception('Insufficient Funds')
    
    def info_all__accounts(self, account_type=''):
        for account in self.__accounts:
            print(f'Number : {account._id}\t Name: {account._name}\t Balance {account._balance}')
    
    def deposit(self, account, amount):
        if self.is_valid_account(account):
            account.deposit(amount)
    

    def withdraw(self, account, amount):
        if self.is_valid_account(account):
            account.withdraw(amount)

    def credit_interest_all(self):
        for account in self.__accounts:
            account.credit_interest()

class ATM: 
    
    def __init__(self, Bank: Bank):
        self.__Bank = Bank

    def __get_user_after_auth(self):
        account_number = int(input('Enter Account Number'))
        password = input('Enter password')
        index, account = self.__Bank.get_account(account_number)
        print(self.__Bank)
        if account :
            if(self.__Bank.authenticate(account, password)):
                return account

            raise Exception('Invalid password')

        raise Exception('Account does not exist')
        

    def check_balance(self):
        account = self.__get_user_after_auth()
        print(f'Available Balance : {account._balance}')

    def cash_money(self):
        account = self.__get_user_after_auth()  
        amount = int(input('Enter Amount to withdraw'))
        account.withdraw(amount)

    


In [3]:
hdfc = Bank()

In [4]:
account1 = hdfc.create_account('Aditi', 'pass1', 52198)
account2 = hdfc.create_account('Aditya', 'pass2',10000)
account3 = hdfc.create_account('Harsh', 'pass3',52198 )


In [5]:
isinstance(account1, object)

True

In [6]:
hdfc.info_all__accounts()

Number : 1	 Name: Aditi	 Balance 52198
Number : 2	 Name: Aditya	 Balance 10000
Number : 3	 Name: Harsh	 Balance 52198


In [7]:
hdfc.transfer_money(account1, account2, 100000)


Exception: Insufficient Funds

In [8]:
hdfc.credit_interest_all()

In [9]:
hdfc.info_all__accounts()

Number : 1	 Name: Aditi	 Balance 52719.98
Number : 2	 Name: Aditya	 Balance 10100.0
Number : 3	 Name: Harsh	 Balance 52719.98


In [10]:
account1.get_info()

'Aditi 52719.98'

In [11]:
hdfcatm = ATM(hdfc)

In [12]:
hdfcatm.check_balance()

ValueError: invalid literal for int() with base 10: ''